In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

## Generate dataframe

In [14]:
class VisualOutcomes:
    
    def vision_list(self, df):
        """
        Returns a list of visual acuity for patient.
        Input: patient Pandas dataframe.
        Output: list.
        """
        df['CreatedDate'] = pd.to_datetime(df['CreatedDate'])
        df.sort_values(by=['CreatedDate'], inplace=True)
        lst = df['visual_acuity'].dropna()
        return lst.to_list()
    
    def mean_vision(self, df):
        """
        Returns the mean vision of a patient.
        Input: patient Pandas dataframe.
        Output: float (LogMAR letters).
        """
        lst = self.vision_list(df)
        return np.mean(lst)

    def loss_from_peak(self, df):
        """
        Returns the VLP for a patient.
        Vision Loss from Peak (VLP) is defined as max vision minus last vision.
        Input: Pandas dataframe.
        Output: float (LogMAR letters).
        """
        visions = self.vision_list(df)
        return max(visions) - visions[-1]
    
    def overall_visual_change(self, df):
        """
        Returns the OVC for a patient.
        Overall Visual Change (OVC) is defined as last vision minus first vision.
        Input: Pandas dataframe.
        Output: float (LogMAR letters).
        """
        visions = self.vision_list(df)
        last = (visions[-1] + visions[-2] + visions[-3]) / 3
        first = visions[0]
        return last - first
    
    def peak_visual_improvement(self, df):
        """
        Returns the PVI for a patient.
        Peak Visual Improvement (PVI) is defined as max vision minus initial vision.
        Input: Pandas dataframe.
        Output: float (LogMAR letters).
        """
        visions = self.vision_list(df)
        return max(visions) - visions[0]

    def proportion_above_baseline(self, df):
        """
        Returns the proportion of time above starting vision.
        Input: patient Pandas dataframe.
        Output: float (percentage).
        """
        lst = self.vision_list(df)
        starting_vision = lst[0]
        above_lst = [i for i in lst if i > starting_vision]
        if len(above_lst) != 0:
            mean = sum(above_lst) / len(above_lst)
        else:
            mean = 0
        return mean

    def patient_clean(self, df, number_years):
        """
        Shortens a patient's dataframe to x years after initiation.
        Input: patient Pandas dataframe, integer.
        Output: Pandas dataframe.
        """
        dates = df['CreatedDate'].to_list()
        first = dates[0]
        cutoff = first.replace(year = first.year + number_years)
        df = df[df['CreatedDate'] < cutoff]
        return df

    def time_above_baseline(self, df, number_years):
        """
        Returns the number of days a patient spent above baseline in first x years.
        Input: Pandas dataframe, integer.
        Output: integer (days).
        """
        df["CreatedDate"] = pd.to_datetime(df["CreatedDate"])
        df.sort_values(by=['CreatedDate'], inplace=True)
        dates = df["CreatedDate"].to_list()
        if (dates[-1] - dates[0]).days > (number_years * 365):
            df = patient_clean(df, number_years)
            vision = df['visual_acuity'].to_list()
            dates2 = df["CreatedDate"].to_list()
            days = 0
            starting_vision = vision[0]
            for i in range(1, len(vision)):
                if vision[i] > vision[0]:
                    between = (dates2[i] - dates2[i-1]).days
                    days += between
            return days
        else:
            return 'nil'
        
    def time_above_baseline2(self, df):
        """
        Returns the number of days a patient spent above baseline in first x years.
        Input: Pandas dataframe, integer.
        Output: integer (days).
        """
        df["CreatedDate"] = pd.to_datetime(df["CreatedDate"])
        df.sort_values(by=['CreatedDate'], inplace=True)
        dates = df["CreatedDate"].to_list()
        vision = df['visual_acuity'].to_list()
        dates2 = df["CreatedDate"].to_list()
        days = 0
        starting_vision = vision[0]
        for i in range(1, len(vision)):
            if vision[i] > vision[0]:
                between = (dates2[i] - dates2[i-1]).days
                days += between
        return days

    def time_to_peak(self, df):
        """
        Returns the TPVI for a patient.
        Time to Peak Visual Improvement (TPVI) is defined in days.
        Input: Pandas dataframe.
        Output: integer (days).
        """
        df["CreatedDate"] = pd.to_datetime(df["CreatedDate"])
        df.sort_values(by=['CreatedDate'], inplace=True)
        dates = df["CreatedDate"].to_list()
        max_value = df['visual_acuity'].max()
        df_fin = df[df['visual_acuity'] == max_value]
        initial_date = dates[0]
        final_date = df_fin.CreatedDate.iloc[0]
        return (final_date - initial_date).days
    
    def baseline_vision(self, df):
        """
        Returns the baseline vision for a patient.
        Input: Pandas dataframe.
        Output: integer (LogMAR letters).
        """
        lst = self.vision_list(df)
        return lst[0]

In [17]:
class Dataframe(VisualOutcomes):
    
    def get_df(self):
        """
        Returns the dataframe to be analysed (all visits).
        """
        df = pd.read_csv('/home/jupyter/charliemacuject/pharma_reports/data/dme.csv')
        df.drop(columns=['Unnamed: 0'], inplace=True)
        return df
    
    def dataframe_gen(self, pdf, pat_id):
        """
        Returns a dataframe of all adherence measures and visual outcomes.
        For a singular patient only (will be one row).
        Input: integer (patient id).
        Output: Pandas dataframe.
        """
        df = pdf[pdf["id"] == pat_id]
        data = {'mean_vision': [VisualOutcomes.mean_vision(self, df)], 
                'time_above_baseline': [VisualOutcomes.time_above_baseline2(self, df)],
                'peak_visual_improvement': [VisualOutcomes.peak_visual_improvement(self, df)],
                'overall_visual_change': [VisualOutcomes.overall_visual_change(self, df)],
                'time_to_peak': [VisualOutcomes.time_to_peak(self, df)],
                'baseline': [VisualOutcomes.baseline_vision(self, df)],
                'visits': [len(df)]}
        return pd.DataFrame(data)

    
    def master_dataframe(self):
        """
        Returns a dataframe of statics for all patients.
        """
        df = self.get_df()
        id_list = df["id"].unique()
        frames = []
        for i in range(len(id_list)):
            try:
                pdf = self.dataframe_gen(df, id_list[i])
                pdf['id'] = i
                frames.append(pdf)
            except:
                i += 1
        master = pd.concat(frames)
        master.reset_index(inplace=True)
        master.drop(columns=['index'], inplace=True)
        return master

In [68]:
dataframe = Dataframe()
df = dataframe.master_dataframe()
df.head()

,mean_vision,time_above_baseline,peak_visual_improvement,overall_visual_change,time_to_peak,baseline,visits,id
0,70.588235,0,0.0,-9.333333,0,76.0,34,0
1,71.687500,54,4.0,-5.666667,328,76.0,18,1
2,73.250000,0,0.0,-26.000000,0,85.0,32,2
3,75.392857,1940,20.0,15.000000,987,65.0,58,3
4,75.600000,50,4.0,-4.000000,363,76.0,17,4


In [69]:
len(df)

181

# Predict patient vision
Goal: predict whether overall visual change (OVC, defined as last vision minus first vision) will be positive or negative for a given patient, using feature-engineered columns that summarise patient clinical history.

First, we need to append a column to the dataframe that will determine whether OVC is positive or negative. To do this, we'll write a function to return a 1 or a 0 based on the value of OVC in that row.

In [70]:
def label_ovc(row): return 1 if row['overall_visual_change'] > 0 else 0

Notice that the parameter being input to `label_ovc` is a Series object named row. Next, we use the `apply` function in Pandas to apply the function. We save the results in a new column in the dataframe.

In [71]:
df['OVC_outcome'] = df.apply(lambda row: label_ovc(row), axis=1)

Let's check that the new column is there.

In [72]:
df.head()

,mean_vision,time_above_baseline,peak_visual_improvement,overall_visual_change,time_to_peak,baseline,visits,id,OVC_outcome
0,70.588235,0,0.0,-9.333333,0,76.0,34,0,0
1,71.687500,54,4.0,-5.666667,328,76.0,18,1,0
2,73.250000,0,0.0,-26.000000,0,85.0,32,2,0
3,75.392857,1940,20.0,15.000000,987,65.0,58,3,1
4,75.600000,50,4.0,-4.000000,363,76.0,17,4,0


Finally, we'll save this amended dataframe so we don't have to run this every time.

In [74]:
df.to_csv('/home/jupyter/charliemacuject/pharma_reports/data/dme_features.csv')

## Baseline logistic regression

In [76]:
import matplotlib.pyplot as plt
from sklearn import linear_model
from scipy.special import expit

df = pd.read_csv('/home/jupyter/charliemacuject/pharma_reports/data/dme_features.csv')

X = df.drop(columns=['overall_visual_change', 'OVC_outcome'])
y = df.OVC_outcome

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2)

clf = linear_model.LogisticRegression(C=1e5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
score = accuracy_score(y_pred, y_test)
print("Accuracy: {}%".format(np.round(100*score,2)))

Accuracy: 86.49%


## Baseline XGBoost

We now need to save our features (`X`) and targets (`y`) by extracting them from the dataframe. Note that we need to drop `overall_visual_change` from the features as well as `OVC_outcome`, as we are classifying OVC rather than regressing it.

In [35]:
X = df.drop(columns=['overall_visual_change', 'OVC_outcome'])
y = df.OVC_outcome

Use scikit-learn to get our train-test split.

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

We now create the XGBoost model using `XGBClassifier`. We don't actually need to specify the objective here, but it's good for clarity. If we had more than two classes, we would instead use `multi:softmax`, but we'll just use a regular logistic function here.

In [41]:
xgb = XGBClassifier(booster='gbtree', objective='binary:logistic', max_depth=6,
                   learning_rate=0.1, n_estimators=100, random_state=2, n_jobs=-1)

Call `fit` on the model to train it.

In [42]:
xgb.fit(X_train, y_train)

[00:21:24] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=2,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

Get the predictions and store them. Let's see how accurate we are.

In [45]:
y_pred = xgb.predict(X_test)
score = accuracy_score(y_pred, y_test)
print("Accuracy: {}%".format(np.round(100*score,2)))

Accuracy: 76.09%


We achieve 76% accuracy with no hyperparameter tuning and minimal feature engineering.

## Hyperparameter tuning

Using a baseline score of 76%, we will see if we can improve upon the classifier using hyperparameter tuning. However, we'll use cross validation accuracy this time to allow us to perform grid searches and random searches.

In [48]:
X = df.drop(columns=['overall_visual_change', 'OVC_outcome'])
y = df.OVC_outcome
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
xgb = XGBClassifier(booster='gbtree', objective='binary:logistic', random_state=2,
                   verbosity=0)

from sklearn.model_selection import cross_val_score
scores = cross_val_score(xgb, X, y, cv=5)
print("Accuracy: ", np.round(scores, 2))
print('Accuracy mean: %0.2f' % (scores.mean()))

Accuracy:  [0.78 0.78 0.78 0.75 0.83]
Accuracy mean: 0.78


When fine-tuning hyperparameters, `GridSearchCV` and `RandomizedSearchCV` are the main options. However, we need to ensure that cross-validation and the searches split the data the same way. Thus, we use stratified k-fold, which includes the same percentage of target values in each fold. 

In [49]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

The `kfold` variable can now be used inside `cross_val_score` and the search functions to ensure consistent results. We'll now redo our baseline using `kfold` so we can appropriately compare.

In [50]:
scores = cross_val_score(xgb, X, y, cv=kfold)
print('Accuracy:', np.round(scores, 2))
print('Accuracy mean: %0.2f' % (scores.mean()))

Accuracy: [0.81 0.75 0.92 0.72 0.75]
Accuracy mean: 0.79


`GridSearchCV` searches all possible combinations in a hyperparameter grid to find the best results. `RandomizedSearchCV` selects 10 random hyperparameter combinations to search, which makes it computationally less expensive than grid search.

We'll write a function that can swap between applying grid search and random search.

In [60]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

def grid_search(params, random=False):
    xgb = XGBClassifier(booster='gbtree', objective='binary:logistic',
                       random_state=2, verbosity=0)
    if random:
        grid = RandomizedSearchCV(xgb, params, cv=kfold, n_iter=20, n_jobs=-1)
    else:
        grid = GridSearchCV(xgb, params, cv=kfold, n_jobs=-1)
    grid.fit(X,y)
    best_params = grid.best_params_
    print("Best params: ", best_params)
    best_score = grid.best_score_
    print("Training score: {:.3f}".format(best_score))

There are many, many hyperparameters we could tune (see the XGBoost documentation [here](https://xgboost.readthedocs.io/en/latest/parameter.html)). We'll choose the key ones.

In [57]:
grid_search(params={'n_estimators':[100, 200, 400, 800]})

Best params:  {'n_estimators': 200}
Training score: 0.801


In [64]:
grid_search(params={'learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]})

Best params:  {'learning_rate': 0.05}
Training score: 0.801


In [ ]:
grid_search(params={'max_depth':[2, 3, 5, 6, 8]})

In [ ]:
grid_search(params={'gamma':[0, 0.1, 0.5, 1, 2, 5]})

In [ ]:
grid_search(params={'min_child_weight':[1, 2, 3, 4, 5]})

In [ ]:
grid_search(params={'subsample':[0.5, 0.7, 0.8, 0.9, 1]})

In [ ]:
grid_search(params={'colsample_bytree':[0.5, 0.7, 0.8, 0.9, 1]})

### Early stopping
Early stopping allows us to choose the number of training rounds without predefining it. Instead, we continue to train until no further gains are produced. A score is determined after each boosting round, and a peak score is achieved when all subsequent scores fail to produce any gains. We choose the number of rounds after peak to examine. 

In [65]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic', random_state=2)
eval_set = [(X_test, y_test)]
eval_metric='error'
model.fit(X_train, y_train, eval_metric="error", eval_set=eval_set, early_stopping_rounds=10, 
          verbose=True)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

[0]	validation_0-error:0.21739
[1]	validation_0-error:0.17391
[2]	validation_0-error:0.19565
[3]	validation_0-error:0.23913
[4]	validation_0-error:0.26087
[5]	validation_0-error:0.26087
[6]	validation_0-error:0.26087
[7]	validation_0-error:0.26087
[8]	validation_0-error:0.23913
[9]	validation_0-error:0.23913
[10]	validation_0-error:0.26087
Accuracy: 82.61%


Early stopping reveals that `n_estimators = 2` gives the best result. We try again, but using a maximum of 5000 trees and 100 consecutive rounds of non-improvement.

In [67]:
model = XGBClassifier(random_state=2, n_estimators=5000)
eval_set = [(X_test, y_test)]
eval_metric="error"
model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set, 
          early_stopping_rounds=100, verbose=False)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 82.61%
